In [2]:
%load_ext autoreload
%autoreload 2

# Getting Started 
Make sure your select *ros_base* as your kernel. You can do this by clicking on the kernel dropdown menu and selecting *ros_base*.


 # Import Python dependencies

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
from IPython.display import Image
import imageio.v2
import csv

import tqdm as tqdm


plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

from iLQR import RefPath
from iLQR import iLQR
# from iLQR import iLQRnp as iLQR


# Task 4: Simulation on Race Tracks

In [ ]:
#@title { run: "auto" , form-width: "30%"}
#@markdown # Set parameters for the planner
#@markdown The default parameter can be used a baseline. Please play around with them to see how they will affect your planned trajectory

# fixed parameters
params = {}
params['L'] = 0.257 # wheelbase [m]
params['m'] = 2.99 # weight [kg]
params['track_width'] = 0.4 # [m]
params['delta_min']= -0.35  # minimum steering angle [rad]
params['delta_max']= 0.35  # maximum steering angle [rad]
params['track_width'] = 0.4 #[m ]

#@markdown ## velocity
v_max = 4 #@param {type:"slider", min:0, max:5, step:0.1}
params['v_max']=v_max # max long vel [m/s]
params['v_min']=0 # min long vel [m/s]


#@markdown ## longitudinal acceleration
a_min = -3.5 #@param {type:"slider", min:-10, max:0, step:0.1}
a_max = 3.5 #@param {type:"slider", min:0, max:10, step:0.1}
params['a_min']= a_min
params['a_max']= a_max

#@markdown ## lateral acceleration
alat_max = 5 #@param {type:"slider", min:0, max:10, step:0.1}
params['alat_max']= alat_max

#@markdown ## weights in cost

w_vel = 2 #@param {type:"number"}
w_contour = 100 #@param {type:"number"}
w_theta =  2#@param {type:"number"}
w_accel =  1#@param {type:"number"}
w_delta = 1 #@param {type:"number"}

params['w_vel'] = w_vel
params['w_contour'] = w_contour
params['w_theta'] = w_theta
params['w_accel'] = w_accel
params['w_delta'] = w_delta

#@markdown ## weights in soft constraints

# parameter for soft constraints
q1_v =  0.4#@param {type:"number"}
q2_v =  2#@param {type:"number"}

q1_road = 0.4#@param {type:"number"}
q2_road = 2#@param {type:"number"}

q1_lat =  0.4#@param {type:"number"}
q2_lat =  2#@param {type:"number"}

params['q1_v'] = q1_v
params['q2_v'] = q2_v
params['q1_road'] = q1_road
params['q2_road'] = q2_road
params['q1_lat'] = q1_lat
params['q2_lat'] = q2_lat

#@markdown ## Disturbance in simulation
sigma_x = 0 #@param {type:"slider", min:0, max:5, step:0.1}
sigma_y = 0 #@param {type:"slider", min:0, max:5, step:0.1}
sigma_v = 0 #@param {type:"slider", min:0, max:5, step:0.1}
sigma_theta = 0 #@param {type:"slider", min:0, max:5, step:0.1}
sigma = np.array([sigma_x, sigma_y, sigma_v, sigma_theta])

#@markdown ## iLQR Parameters
T = 2 #@param {type:"number"}
N = 10 #@param {type:"integer"}
max_itr = 50  #@param {type:"integer"}
params['T'] = T
params['N'] = N+1
params['max_itr'] = max_itr

track_name = "Austin" #@param ["Austin", "BrandsHatch", "Budapest", "Catalunya", "Hockenheim", "IMS", "MexicoCity", "Montreal", "Monza", "MoscowRaceway", "Norisring", "Nuerburgring", "Oschersleben", "Sakhir", "SaoPaulo", "Sepang", "Shanghai", "Silverstone", "Sochi", "Spa", "Spielberg", "Suzuka", "YasMarina", "Zandvoort"]

In [ ]:
#@markdown # Run Simulation

dt = params['T'] / (params['N']-1)

def simulate_kin(state, control, step = 10):

    accel = np.clip(control[0], params['a_min'], params['a_max'])
    delta = np.clip(control[1], params['delta_min'], params['delta_max'])
    
    next_state = state
    dt_step = dt/step
    for _ in range(step):
        # State: [x, y, psi, v]
        d_x = (next_state[2]*dt_step+0.5*accel*dt_step**2)*np.cos(next_state[3])
        d_y = (next_state[2]*dt_step+0.5*accel*dt_step**2)*np.sin(next_state[3])
        d_v = accel*dt_step
        d_psi = (next_state[2]*dt_step+0.5*accel*dt_step**2)*np.tan(delta)/params['L']
        next_state = next_state + np.array([d_x, d_y, d_v, d_psi])+sigma*np.random.normal(size=(4))*dt_step
        next_state[2] = max(0, next_state[2])
    return next_state

def load_track(track_name):
  x = []
  y = []
  width_r = []
  width_l = []
  filename = 'racetrack/'+track_name+'.csv'
  with open(filename, newline='') as f:
      spamreader = csv.reader(f, delimiter=',')
      for i, row in enumerate(spamreader):
          if i>0:
              x.append(float(row[0]))
              y.append(float(row[1]))
              width_r.append(float(row[2]))
              width_l.append(float(row[3]))

  x = np.array(x)/25.0
  y = np.array(y)/25.0

  x_range = max(x) - min(x)
  y_range = max(y) - min(y)
  ratio = x_range/y_range
  if ratio < 1:
    center_line = np.array([y,x])
    ratio = 1/ratio
  else:
    center_line = np.array([x,y])
  track = Track(center_line = center_line, width_left = 0.4, width_right = 0.4)

  
  return track, ratio

# initialize the ilqr
track, ratio = load_track(track_name)
solver = iLQR(track, params)

itr_max = 500
state_hist = np.zeros((4,itr_max+1))
control_hist = np.zeros((2,itr_max))
plan_hist = np.zeros((4, params['N'], itr_max))

pos0, psi0 = track.interp([0])
x_cur =np.array([pos0[0], pos0[1], 0, psi0[0]])
init_control = np.zeros((2,params['N']))
 
t_total = 0

# solve
for i in range(itr_max):
    
    states, controls, t_process, status, theta \
            = solver.solve(x_cur, controls = init_control) 
    
    plan_hist[:,:, i] = states
    
    state_hist[:,i] = states[:,0]
    control_hist[:,i] = controls[:,0]
    init_control[:,:-1] = controls[:,1:]
    
    x_cur = simulate_kin(x_cur, controls[:,0])
    t_total += t_process
   
    
    if theta[1]<theta[0]:
        break

state_hist = state_hist[:, :i+2]
control_hist = control_hist[:,:i+1]
plan_hist = plan_hist[:,:, :i+1]
print("Complete a lap on ", track_name, " in ", (i+1)*dt, 'secs')

plot_zoom = 1 

# plot parameter
fig_x_max = 20*plot_zoom
fig_y_max = 10*plot_zoom
cut_off_ratio = fig_x_max/fig_y_max
if ratio>=cut_off_ratio: #
  fig_x = fig_x_max
  fig_y = fig_x_max/ratio
else:
  fig_y = fig_y_max
  fig_x = fig_y_max*ratio

In [ ]:
#@title { run: "auto" }

#@markdown ## Scale the plot to fit your screen
plot_zoom = 0.6 #@param {type:"slider", min:0.1, max:2, step:0.1}

# plot parameter
fig_x_max = 30*plot_zoom
fig_y_max = 15*plot_zoom
cut_off_ratio = fig_x_max/fig_y_max
if ratio>=cut_off_ratio: #
  fig_x = fig_x_max
  fig_y = fig_x_max/ratio
else:
  fig_y = fig_y_max
  fig_x = fig_y_max*ratio

In [ ]:
#@markdown # Visualize simulation (This takes a while to generate)
sim = plt.figure(figsize=(fig_x, fig_y))
N = plan_hist.shape[-1]
track.plot_track()
plan_plot = plt.plot([], [], linewidth= 4)[0]
traj_plot = plt.scatter([], [], s = 80, c=[], cmap=cm.jet, vmin=0, vmax=params['v_max'], edgecolor='none', marker='o')
cbar = plt.colorbar(traj_plot)
cbar.set_label(r"velocity [$m/s$]", size=20)
plt.axis('equal')
plt.close()

def drawframe(n):
    traj_plot.set_offsets(state_hist[:2,:n+1].T)
    traj_plot.set_array(state_hist[2,:n+1])
    plan_plot.set_data(plan_hist[0,:, n], plan_hist[1,:, n])
    
    return (traj_plot,plan_plot)


# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(sim, drawframe, frames=N, interval=dt*1000, blit=True)
HTML(anim.to_html5_video())

In [ ]:
#@markdown  ## Plot raceline
plt.figure(figsize=(fig_x, fig_y))
plt.plot(state_hist[0,:], state_hist[1,:], 'b-', linewidth= 5, alpha =0.6, label = 'Trajectory')
track.plot_track(centerline = True)
plt.legend(prop={'size': 20})
#plt.title("Trajectory", size = 20)
plt.axis('equal')
plt.axis('off')
plt.show()





In [ ]:
#@markdown  ## Visualize velocity along the track
plt.figure(figsize=(fig_x, fig_y))
track.plot_track()
sc = plt.scatter(state_hist[0, :-1], state_hist[1,:-1], s = 80, 
                c=state_hist[2,:-1], cmap=cm.jet, 
                vmin=params['v_min'], vmax=params['v_max'], edgecolor='none', marker='o')
cbar = plt.colorbar(sc)
cbar.set_label(r"Velocity [$m/s$]", size=20)
plt.axis('equal')
plt.show()

In [ ]:
#@markdown  ## Visualize longitudinal acceleration along the track
plt.figure(figsize=(fig_x, fig_y))
track.plot_track()
sc = plt.scatter(state_hist[0, :-1], state_hist[1,:-1], s = 80, 
                c=control_hist[0,:], cmap=cm.jet, 
                vmin=params['a_min'], vmax=params['a_max'], edgecolor='none', marker='o')
cbar = plt.colorbar(sc)
cbar.set_label(r"Longitudinal Accel [$m/s^2$]", size=20)
plt.axis('equal')
plt.show()

In [ ]:
#@markdown  ## Visualize lateral acceleration along the track
plt.figure(figsize=(fig_x, fig_y))
track.plot_track()
alat = state_hist[2,:-1]**2*np.tan(control_hist[1,:])/0.257
sc = plt.scatter(state_hist[0, :-1], state_hist[1,:-1], s = 80, 
                c=alat, cmap=cm.jet, 
                vmin = 0, vmax=params['alat_max'], edgecolor='none', marker='o')
cbar = plt.colorbar(sc)
cbar.set_label(r"Lateral Accel [$m/s^2$]", size=20)
plt.axis('equal')
plt.show()